<a href="https://colab.research.google.com/github/garrer/NTERSOL_DE/blob/main/Copy_of_SparkTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Code required to get Spark environment

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 199 kB 58.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=4e07733c63f0e26cdd75e932a411a8a69d01ce379bbd44ea87dee94ca87dd16f
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [4]:
spark

In [18]:
raw_df = spark.read.csv('sample_data/data-engineer-interview-data.csv', header=True)

In [19]:
raw_df.createOrReplaceTempView("TitleData")

In [6]:
spark.sql("select * from TitleData limit 10")

DataFrame[DocumentNumber: string, DocumentDate: string, DocumentType: string, RefersToDocumentNumber: string, RefersToDocumentYear: string, Remarks: string]

In [7]:
raw_df.show(10)

+--------------+------------+------------+----------------------+--------------------+------------+
|DocumentNumber|DocumentDate|DocumentType|RefersToDocumentNumber|RefersToDocumentYear|     Remarks|
+--------------+------------+------------+----------------------+--------------------+------------+
|            27|   8/16/2021|           C|                  null|                null|        null|
|             1|   2/15/2021|           T|                     8|                2020|2020 0008-00|
|            67|   10/9/2020|           A|                  null|                null|        null|
|           157|   10/9/2020|           A|                  null|                null|        null|
|           189|   10/9/2020|           J|                  null|                null|        null|
|           250|   10/9/2020|           R|                  2016|                  98|        null|
|            87|    8/7/2020|           R|                  1992|                  97|        null|


In [22]:
spark.sql ("""
select DocumentNumber,DocumentDate, Documenttype, RefersToDocumentNumber,RefersToDocumentYear
, concat (DocumentNumber,split(DocumentDate,'/')[2]) as document_unique_id
, row_number() OVER (PARTITION BY concat (DocumentNumber,split(DocumentDate,'/')[2]) ORDER BY DocumentNumber) as rn
, CASE WHEN length (RefersToDocumentYear) < 4 THEN concat (RefersToDocumentYear,RefersToDocumentNumber) ELSE concat (RefersToDocumentNumber,RefersToDocumentYear) END as document_ref_unique_id
from TitleData
""").createOrReplaceTempView("derived")

In [8]:
# spark.sql("""
               
               
               
#                 select *, CAST ((DocumentNumber+split(DocumentDate,'/')[2]) AS INT) as document_unique_Id,  
#                     CAST ( COALESCE(RefersToDocumentNumber, 0)+ COALESCE (RefersToDocumentYear, 0) AS INT) as document_ref_unique_Id                   
#                     from TitleData
#                     """).createOrReplaceTempView("derived")

In [23]:
spark.sql("""select * from derived limit 10""").show(10)

+--------------+------------+------------+----------------------+--------------------+------------------+---+----------------------+
|DocumentNumber|DocumentDate|Documenttype|RefersToDocumentNumber|RefersToDocumentYear|document_unique_id| rn|document_ref_unique_id|
+--------------+------------+------------+----------------------+--------------------+------------------+---+----------------------+
|           100|   8/21/2002|           J|                  null|                null|           1002002|  1|                  null|
|           100|  12/24/2010|           B|                  null|                null|           1002010|  1|                  null|
|           100|    2/5/2010|           C|                  null|                null|           1002010|  2|                  null|
|           101|  11/29/2002|           A|                  null|                null|           1012002|  1|                  null|
|            10|   7/18/1988|           B|                  null|    

In [24]:
spark.sql ("""select t1.*, t2.DocumentType as ReferenceType           
                 from derived t1 LEFT JOIN derived t2 on t1.document_ref_unique_Id = t2.document_unique_Id"""            
               ).createOrReplaceTempView("derived_final")

In [26]:
spark.sql("""select * from derived_final limit 10""").show(10)

+--------------+------------+------------+----------------------+--------------------+------------------+---+----------------------+-------------+
|DocumentNumber|DocumentDate|Documenttype|RefersToDocumentNumber|RefersToDocumentYear|document_unique_id| rn|document_ref_unique_id|ReferenceType|
+--------------+------------+------------+----------------------+--------------------+------------------+---+----------------------+-------------+
|           100|   8/21/2002|           J|                  null|                null|           1002002|  1|                  null|         null|
|           100|  12/24/2010|           B|                  null|                null|           1002010|  1|                  null|         null|
|           100|    2/5/2010|           C|                  null|                null|           1002010|  2|                  null|         null|
|           101|  11/29/2002|           A|                  null|                null|           1012002|  1|         

In [23]:
%%markdown

* Documents of type J (a "judgment" document) no longer apply if a document of type T (a "termination" document) refers to it
* Documents of any other type no longer apply if a document of type R (a "release" document) refers to it




* Documents of type J (a "judgment" document) no longer apply if a document of type T (a "termination" document) refers to it
* Documents of any other type no longer apply if a document of type R (a "release" document) refers to it



In [28]:
spark.sql ("""
            
            SELECT * FROM derived_final t1 WHERE t1.document_unique_Id NOT IN (
            SELECT  document_unique_Id  FROM derived_final t1 where (lower (t1.DocumentType) == 't' and
            lower (t1.ReferenceType) == 'j') OR (lower (t1.ReferenceType) == 'r')) 
                
            """).createOrReplaceTempView("TitleDataFinal")

In [29]:
#test case: Check if TitleDataFinal has any document type J referencing to document type T
spark.sql (""" select * from TitleDataFinal where upper(DocumentType)=='T' and upper(ReferenceType)=='J' """).show(10)

+--------------+------------+------------+----------------------+--------------------+------------------+---+----------------------+-------------+
|DocumentNumber|DocumentDate|Documenttype|RefersToDocumentNumber|RefersToDocumentYear|document_unique_id| rn|document_ref_unique_id|ReferenceType|
+--------------+------------+------------+----------------------+--------------------+------------------+---+----------------------+-------------+
+--------------+------------+------------+----------------------+--------------------+------------------+---+----------------------+-------------+



In [30]:
#test case: Check if TitleDataFinal has any record referencing document type R
spark.sql (""" select * from TitleDataFinal where upper(ReferenceType)=='R' """).show(10)

+--------------+------------+------------+----------------------+--------------------+------------------+---+----------------------+-------------+
|DocumentNumber|DocumentDate|Documenttype|RefersToDocumentNumber|RefersToDocumentYear|document_unique_id| rn|document_ref_unique_id|ReferenceType|
+--------------+------------+------------+----------------------+--------------------+------------------+---+----------------------+-------------+
+--------------+------------+------------+----------------------+--------------------+------------------+---+----------------------+-------------+



In [32]:
spark.sql (""" select count(*) from TitleDataFinal """).show(10)

+--------+
|count(1)|
+--------+
|     142|
+--------+

